In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas.tools.plotting import scatter_matrix

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.linear_model import LogisticRegression#, LinearRegression

In [3]:
accounts = pd.read_csv('data/MFG-accounts.csv')

In [4]:
customers = pd.read_csv('data/MFG-customers.csv')

In [5]:
invoices = pd.read_csv('data/MFG-invoices.csv')

In [6]:
industry = pd.read_csv('data/MFG-industry.csv')

In [7]:
press_release = pd.read_csv('data/Press_Release.csv')

In [8]:
zip_codes = pd.read_csv('data/zip_code_database.csv')

In [9]:
zip_codes = zip_codes[['zip','state']]

### Accounts

In [10]:
label_df = pd.DataFrame(accounts['Account_Description'].unique())
label_df.columns = ['Account_Description']

In [11]:
label_list = [
    'scanning',
    'fdm',
    'carbon',
    'conlas',
    'manufacturing',
    'amermakes',
    'rent_inc_mfg',
    'sla',
    'molding',
    'casting',
    'sls',
    'polyjet',
    'metal',
    'freight',
    'cgb',
    'cgb',
    'misc',
    'asset'
]

In [12]:
# typedf.merge(pd.DataFrame(label_list),left_index = True, right_index = True, how = 'left')
label_df['label'] = label_list

In [13]:
accounts_labels = accounts.merge(label_df, left_on = 'Account_Description', right_on = 'Account_Description', how = 'left')

In [14]:
accounts_dummies = pd.get_dummies(accounts_labels, columns=['label'])


### Customers

In [15]:
customers['Country'] = customers['Country'].replace(['UNITED STATES', 'United States'], ['USA', 'USA',])
customers['Country'] = customers['Country'].fillna('USA')
customers['CustID'] = customers['CustomerID']
customers['State'] = customers['State'].replace('az', 'AZ')
customers['State'] = customers['State'].replace('Te','TX')
customers['State'] = customers['State'].fillna('FO') #FO = not US state, foreign 

In [16]:
def strip_zip(longZip):
    value = str(longZip).split('-')[0]
    return value

In [17]:
#customers['Zip'] = customers['Zip'].str.replace('-','')#.astype(int)
#customers['Zip'].unique()

In [18]:
customers['zipcode'] = customers['Zip'].apply(strip_zip)

In [19]:
customer_location = customers[['CustID', 'State','zipcode','Country']]

In [46]:
pd.options.display.max_rows = 999

In [21]:
customer_state_dummies = pd.get_dummies(customer_location, columns=['State'])

In [63]:
customer_region_dummies = pd.get_dummies(customer_state_dummies, columns=['Country'])

### Invoices

In [23]:
invoices['datetime'] = pd.to_datetime(invoices['Date'])

In [24]:
invoices['dayofweek'] = invoices['datetime'].dt.dayofweek

In [25]:
invoices ['Amount'] = (invoices['Amount'].str.replace(',', '').astype(float)).abs()

invoices['Qty'] = invoices['Qty'].str.replace(',','').astype(float)
invoices['Qty'] = invoices['Qty'].fillna(0)

invoices['Account_ID'] = invoices['Account_ID'].str.replace('-','').astype(int)
#invoices['Item_ID'] = invoices['Item_ID'].str.replace('-','')

invoices ['Unit_Price'] = invoices['Unit_Price'].str.replace(',','').astype(float)
invoices ['Unit_Price'] = invoices['Unit_Price'].fillna(0)


In [26]:
repeat=[]
for invoice in invoices.iterrows():
    repeat.append(((invoice[1]['CustID'] == invoices['CustID']) &
    (invoice[1]['datetime'] < invoices['datetime']) &
    (invoice[1]['datetime']  + pd.Timedelta(365, 'd') > invoices['datetime'])).sum()> 0)  

In [27]:
invoices['repeat'] = repeat

In [28]:
def year(col):
    value = int(str(col)[0:4])
    return value
    
def month(col):
    value = int(str(col)[8:11])
    return value

In [29]:
invoices['year'] = invoices['datetime'].apply(year)
invoices['month'] = invoices['datetime'].apply(month)

In [30]:
def quarters(months):
    
    quarters=[]
    for month in months:
        if month>= 1 and month<=3:
            quarters.append(1)
        elif month >= 4 and month <= 6:
            quarters.append(2)
        elif month >= 7 and month<=9:
            quarters.append(3)
        elif month >= 10 and month >= 12:
            quarters.append(4)
        else:
            quarters.append(0)
    return quarters


In [31]:
invoices['quarter']= quarters(invoices['month'])

In [32]:
#invoices.loc[:20,['month','quarter']]

In [33]:
invoices['Item_ID'] = invoices['Item_ID'].fillna('other')

In [34]:
invoices['repeat_c']= invoices['repeat'].replace('True','1').astype(int)
invoices['repeat_c']= invoices['repeat'].replace('False','0').astype(int)

In [ ]:
invoices.group_by('CustID').aggregate({'datetime'
                                      pri
                                      
                                      
                                      })

In [35]:
invoices_items_dummies = pd.get_dummies(invoices, columns=['Item_ID'])

In [36]:
# don't need price as a dummy.
#invoices_items_prices_dummies = pd.get_dummies(invoices_items_dummies, columns=['unit_price'])#

In [37]:
invoices_items_year_dummies = pd.get_dummies(invoices_items_dummies, columns=['year'])

In [38]:
invoices_items_year_month_dummies = pd.get_dummies(invoices_items_year_dummies, columns=['month'])

In [39]:
invoices_items_year_month_quarter_dummies = pd.get_dummies(invoices_items_year_month_dummies, columns=['quarter'])

In [40]:
final_invoices = invoices_items_year_month_quarter_dummies


In [ ]:
#final_invoices

### press_release

In [41]:
press_release['release_date'] = pd.to_datetime(press_release['Release Date'])

In [42]:
press_release ['Premium'] = press_release['Premium'].str.replace('-', 'free')
press_release['Premium'] = press_release['Premium'].fillna('Premium')

In [43]:
press_release['headline_len'] = press_release['Headline'].str.len()

### Industries

In [44]:
industry.columns

Index(['CustID', 'Industry'], dtype='object')

In [47]:
industry_dummies = pd.get_dummies(industry, columns =['Industry'])

In [49]:
#industry_dummies.head()

In [ ]:
final_invoices 

### merge invoices industry region

In [53]:
invoices_industry_dummies = pd.merge(final_invoices, industry_dummies, how='left', on=['CustID'])

In [64]:
invoices_industry_location_dummies = pd.merge(invoices_industry_dummies, customer_region_dummies, how='left', on=['CustID'])

In [65]:
invoices_industry_location_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13349 entries, 0 to 13348
Columns: 214 entries, Date to Country_USA
dtypes: bool(1), datetime64[ns](1), float64(3), int64(4), object(2), uint8(203)
memory usage: 3.7+ MB


In [75]:
list(invoices_industry_location_dummies.columns)

['Date',
 'Qty',
 'Unit_Price',
 'Account_ID',
 'Amount',
 'CustID',
 'datetime',
 'dayofweek',
 'repeat',
 'repeat_c',
 'Item_ID_RP-1-222-0502',
 'Item_ID_RP-1-222-0503',
 'Item_ID_RP-1001717',
 'Item_ID_RP-1001718',
 'Item_ID_RP-1513836',
 'Item_ID_RP-1859701',
 'Item_ID_RP-19112-23',
 'Item_ID_RP-19112-24',
 'Item_ID_RP-19112-25',
 'Item_ID_RP-19112-26',
 'Item_ID_RP-19112-51',
 'Item_ID_RP-19112-52',
 'Item_ID_RP-19112-57',
 'Item_ID_RP-19112-59',
 'Item_ID_RP-19112-61',
 'Item_ID_RP-19112-62',
 'Item_ID_RP-2718001',
 'Item_ID_RP-300727A',
 'Item_ID_RP-300784A',
 'Item_ID_RP-301550-01',
 'Item_ID_RP-301550-02',
 'Item_ID_RP-3DSCAN-1',
 'Item_ID_RP-AMP-1',
 'Item_ID_RP-C10047',
 'Item_ID_RP-C10048',
 'Item_ID_RP-C10049',
 'Item_ID_RP-CONSULT-1',
 'Item_ID_RP-CS007200 A/B',
 'Item_ID_RP-CS007650',
 'Item_ID_RP-DLS-1',
 'Item_ID_RP-FDM-1',
 'Item_ID_RP-FM002690',
 'Item_ID_RP-FM002868',
 'Item_ID_RP-FM002869',
 'Item_ID_RP-INJMOLD-1',
 'Item_ID_RP-METAL-1',
 'Item_ID_RP-MFG-1',
 'Item

### extras


In [ ]:
invoices_industry ['dummy_itemid'] = (invoices_industry['Item_ID'].replace(['RP-SLA-1', 'SHP-MFG',
                                                        'RP-INJMOLD-1', 'RP-FDM-1', 'RP-SLS-1',
       'RP-OBJ-1', 'RP-RTV-1', 'RP-3DSCAN-1', 'RP-MFG-1', 'RP-MTL0010',
       'RP-MTL0011', 'RP-MTL0015', 'RP-PLS0033', 'RP-PLS0034',
       'RP-PLS0035', 'RP-PLS0036', 'RP-PLS0037', 'RP-PLS0038',
       'RP-PLS0039', 'RP-PLS0040', 'RP-PLS0041', 'RP-PLS0042',
       'RP-PLS0043', 'RP-PLS0028', 'RP-PLS0029', 'RP-PLS0030',
       'RP-FM002868', 'RP-FM002869', 'RP-MTL0014', 'RP-FM002690',
       'RP-1513836', 'RP-1859701', 'RP-PLS0025', 'RP-PLS0026',
       'RP-PLS0027', 'RP-19112-51', 'RP-19112-23', 'RP-19112-24',
       'RP-19112-52', 'RP-19112-57', 'RP-19112-61', 'RP-CS007200 A/B',
       'RP-CS007650', 'RP-19112-25', 'RP-19112-26', 'RP-19112-59',
       'RP-19112-62', 'RP-AMP-1', 'RP-C10047', 'RP-C10048', 'RP-C10049',
       'RP-301550-01', 'RP-301550-02', 'RP-300727A', 'RP-300784A',
       'RP-1-222-0502', 'RP-1-222-0503', 'RP-METAL-1', 'RP-2718001',
       'RP-DLS-1', 'RP-CONSULT-1', 'RP-1001717', 'RP-1001718'],
        [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
        31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,
        58,59,60,61,62,63]))

In [ ]:
invoices_industry ['dummy_itemid'] = invoices_industry['dummy_itemid'].fillna(90)

In [ ]:
invoices_industry ['dummy_state'] = (invoices_industry['State'].replace(['CO', 'AZ',
        'UT', 'CA', 'MA', 'OH', 'NJ', 'PA', 'FL', 'IL',
       'TN', 'NE', 'MD', 'IN', 'WA', 'NC', 'GA', 'NM', 'MI', 'WI', 'NV',
       'NH', 'Te', 'TX', 'KS', 'MN', 'SC', 'MO', 'NS', 'OR', 'VA', 'LA',
       'ON', 'NY', 'BC'],[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
                                31,32,33,34,35]))

In [ ]:
invoices_industry['dummy_country']= (invoices_industry['Country'].replace(['USA', 'Germany', 'Costa Rica', 'China', 'Canada', 'Philippines'],
                                                                          [1,2,3,4,5,6]))

In [ ]:
invoices_industry ['industry1'] = invoices_industry['Industry']
invoices_industry ['industry1'] = (invoices_industry['industry1'].replace(['Individual', 'Consumer Product', 'Manufacturing Equipment',
       'Design', 'Antenna', 'Automotive', 'Education', 'Manufacturing',
       'Medical', 'Electronics', 'HVAC', 'Engineering', 'Machinary',
       'Unknown', 'Telecomunications', 'Firearms', 'Swimming Pools',
       'Defence', 'Aerospace', 'Government', 'Injection Molding',
       'Distributor', 'Sensors', 'Testing  ', 'Lighting',
       'Water Treatment', 'Mining', 'Internal', 'Components',
       'Machine Shop', 'Power', 'Alternative Energy', 'Other',
       'Construction', 'Sporting Goods', '3D Printing Service Provider',
       'Semiconductor Equipment', 'Building Products', 'Architecture',
       'Machine shop', 'Clothing', 'Additive Manufacturing',
       'Agriculture Equip', 'Oil & Gas', 'Chemicals', 'Food &Beverage',
       'Turbomachinary', 'Solar', 'Materials', 'Utility', 'Robotics',
       'Furniture', 'Packaging', 'Batteries', 'Printing',
       'Transportation', 'IT'], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
                                31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57]))

#invoices_industry ['industry1'] 

### Logistic regretios


In [ ]:
y = invoices_industry_location_dummies['repeat_c'] == True
X = (invoices_industry_location_dummies[['year', 'month', 'unit_price', 'qty', 'account_id', 'amount','industry1',
                        'dummy_country','dummy_state','dummy_itemid']])

In [ ]:
model1 = LogisticRegression()

In [ ]:
model1.fit(X, y)

In [ ]:
model1.coef_

In [ ]:
#model.predict_proba([[4, 10]])

In [ ]:
y = invoices_industry['repeat_c'] == True
x = (invoices_industry[['year']])

fig, ax = plt.subplots()
ax.scatter(x, y)

### Random Fores

In [71]:
from sklearn.model_selection import train_test_split

In [72]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

In [74]:

y = pd.Series(invoices_industry_location_dummies.repeat_c)

X = (invoices_industry_location_dummies [[]] )

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.33,
                                                    random_state=2)



ValueError: cannot copy sequence with size 13349 to array axis with dimension 214

In [ ]:
# Parameter Search                                     
model = DecisionTreeClassifier(max_depth=5)

model.fit(X_train,y_train)
print(model.score(X_test, y_test))

In [ ]:
# Parameter Search                                     
model = DecisionTreeClassifier()
depth_parm = np.arange(1, 12, 1)
num_samples_parm = np.arange(5,95,10)
parameters = {'max_depth' : depth_parm,
             'min_samples_leaf' : num_samples_parm}
clf = GridSearchCV(model, parameters, cv=10, n_jobs=-1)
clf.fit(X_train,y_train)
print(clf.score(X_test, y_test))

In [ ]:
y_train.mean()

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_params_

In [ ]:
# Train and fit model                                                   
rf = RandomForestClassifier(n_estimators=1000,
                           max_features='auto',
                           random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)
                                     
# Test Prediction
pred = rf.predict(X_test)
print(rf.score(X_test, y_test))